In [0]:
# FUNÇÕES UTILIZADAS ==> (groupBy) / (date_format) / (when + otherwise = IF else) / (cast) / (substring)

In [0]:
# IMPORTANDO BIBLIOTECAS 

import pyspark.sql.functions
from pyspark.sql.types import StructType, StructField , StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local(*)")
        .appName('pyspark-3')
        .getOrCreate()
        )



In [0]:
dados = (
    ['1','José','Anápolis','São Paulo','01-09-1900'],
    ['2','Igor','Anápolis','São Paulo','11-09-1977'],
    ['3','Leonardo','Anápolis','São Paulo','21-12-2000'],
    ['4','Humberto','Pato Branco','Rio Grande do Sul','13-11-1964'],
    ['5','Isaías','Pato Branco','Rio Grande do Sul','07-07-2002'],
    ['6','Lucas','Tauá','Ceará','05-09-1984']
    
)

In [0]:
# CRIANDO COLUNAS SCHEMA DE DADOS ==> STRUCT TYPE 

colunas = (StructType([
    StructField('cod_cliente' , StringType(), True),
    StructField('nome' , StringType(), True),
    StructField('municipio' , StringType(), True),
    StructField('estado' , StringType(), True),
    StructField('data_nasc' , StringType(), True),
]))

In [0]:
# CRIANDO DATAFRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

df.display()

cod_cliente,nome,municipio,estado,data_nasc
1,José,Anápolis,São Paulo,01-09-1900
2,Igor,Anápolis,São Paulo,11-09-1977
3,Leonardo,Anápolis,São Paulo,21-12-2000
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002
6,Lucas,Tauá,Ceará,05-09-1984


In [0]:
# 1) EXIBA A QNT DE MUNICIPIOS (GROUP BY + COUNT)

df.groupBy('municipio').count().show()


+-----------+-----+
|  municipio|count|
+-----------+-----+
|   Anápolis|    3|
|Pato Branco|    2|
|       Tauá|    1|
+-----------+-----+



In [0]:
# 2) CRIE UMA COLUNA (IDADE) A PARTIR DA DATA ATUAL - A DATA_NASC 

df = df.withColumn('idade', date_format(current_timestamp() , 'yyyy').cast('integer') - substring('data_nasc' , 7,4).cast('integer'))

df.show()



+-----------+--------+-----------+-----------------+----------+-----+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|
+-----------+--------+-----------+-----------------+----------+-----+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|  122|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|   45|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|   22|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|   58|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|   20|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|   38|
+-----------+--------+-----------+-----------------+----------+-----+



In [0]:
# 3) CRIE UMA COLUNA (STATUS ) ==> SE A PESSOA FOR MAIOR DE 22 ESSA COLUNA VAI FICAR (OK)

# FUNCAO IF E ELSE ==> IF = WHEN     ;  ELSE = OTHERWISE

df = df.withColumn("status" , when(col("idade") >= 22 , "OK").otherwise("NOT"))

df.display()

cod_cliente,nome,municipio,estado,data_nasc,idade,status
1,José,Anápolis,São Paulo,01-09-1900,122,OK
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT
6,Lucas,Tauá,Ceará,05-09-1984,38,OK


In [0]:
# 4) CRIE UMA COLUNA (PAULISTA) E INFORME SE A PESSOA É OU NÃO PAULISTA 

df = df.withColumn('Paulista' , when(col('estado') == 'São Paulo' , "Sim").otherwise('Não'))

df.display()




cod_cliente,nome,municipio,estado,data_nasc,idade,status,Paulista
1,José,Anápolis,São Paulo,01-09-1900,122,OK,Sim
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK,Sim
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK,Sim
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK,Não
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT,Não
6,Lucas,Tauá,Ceará,05-09-1984,38,OK,Não


In [0]:
# 5) FUNÇÃO FILTER ==> ENCONTRE OS QUE (NÃO) SÃO PAULISTA E O ESTADO SEJA CEARÁ 

df.filter(col('Paulista') == 'Não').filter(col('estado') == 'Ceará').show()

+-----------+-----+---------+------+----------+-----+------+--------+
|cod_cliente| nome|municipio|estado| data_nasc|idade|status|Paulista|
+-----------+-----+---------+------+----------+-----+------+--------+
|          6|Lucas|     Tauá| Ceará|05-09-1984|   38|    OK|     Não|
+-----------+-----+---------+------+----------+-----+------+--------+



In [0]:
# 6) ACHE TODOS QUE (NÃO) SÃO (PAULISTAS)

df.filter(col('Paulista') == 'Não').show()

+-----------+--------+-----------+-----------------+----------+-----+------+--------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|status|Paulista|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|   58|    OK|     Não|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|   20|   NOT|     Não|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|   38|    OK|     Não|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+



In [0]:
#7) ENCONTRE TODOS QUE SÃO PAULISTA COM IDADE MAIOR QUE 22 

df.filter(col('idade') > 22).filter(col('Paulista') == 'Sim').show()

+-----------+----+---------+---------+----------+-----+------+--------+
|cod_cliente|nome|municipio|   estado| data_nasc|idade|status|Paulista|
+-----------+----+---------+---------+----------+-----+------+--------+
|          1|José| Anápolis|São Paulo|01-09-1900|  122|    OK|     Sim|
|          2|Igor| Anápolis|São Paulo|11-09-1977|   45|    OK|     Sim|
+-----------+----+---------+---------+----------+-----+------+--------+



In [0]:
# 8) NA COLUNA (DATA_NASC) TROQUE O SINAL (-) por (/)

# OBS ==> (REGEPX_REPLACE) ==> NA MAIORIA DAS VEZES SÓ FUNCIONA COM (withColumn) => não é bom colocar (filter;where)

df = df.withColumn('data_nasc', regexp_replace('data_nasc' , '\-' , '/'))  # ESSA (\) => significa que ira pegar todos sinais

df.show()

# OBS => NÃO PODE FAZER DESTE JEITO ABAIXO, POIS ELE NÃO PERCORRE CARACTER POR CARACTER ==> ELE BUSCA A PALAVRA COMPLETA

#df = df.withColumn('data_nasc', when(col('data_nasc') == '-' , "/").otherwise('data_nasc')).show()


+-----------+--------+-----------+-----------------+----------+-----+------+--------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|status|Paulista|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+
|          1|    José|   Anápolis|        São Paulo|01/09/1900|  122|    OK|     Sim|
|          2|    Igor|   Anápolis|        São Paulo|11/09/1977|   45|    OK|     Sim|
|          3|Leonardo|   Anápolis|        São Paulo|21/12/2000|   22|    OK|     Sim|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13/11/1964|   58|    OK|     Não|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07/07/2002|   20|   NOT|     Não|
|          6|   Lucas|       Tauá|            Ceará|05/09/1984|   38|    OK|     Não|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+



In [0]:
df.printSchema()

root
 |-- cod_cliente: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- status: string (nullable = false)
 |-- Paulista: string (nullable = false)



In [0]:
# 9) Crie uma coluna chamada (ADULTO) ==> para os que tem idade maior que 22 

df = df.withColumn('Adulto' , when(col('idade') > 22 , lit('sim') ).otherwise('não'))

# df = df.withColumn('Adulto' , when(col('idade') > 22 , 'sim' ).otherwise('não')) # => tambem funciona

df.show()

+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|status|Paulista|Adulto|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+
|          1|    José|   Anápolis|        São Paulo|01/09/1900|  122|    OK|     Sim|   sim|
|          2|    Igor|   Anápolis|        São Paulo|11/09/1977|   45|    OK|     Sim|   sim|
|          3|Leonardo|   Anápolis|        São Paulo|21/12/2000|   22|    OK|     Sim|   não|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13/11/1964|   58|    OK|     Não|   sim|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07/07/2002|   20|   NOT|     Não|   não|
|          6|   Lucas|       Tauá|            Ceará|05/09/1984|   38|    OK|     Não|   sim|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+

